<a href="https://colab.research.google.com/github/Giraud-Pierre/DeepLearning_FineTuneLLama2Project/blob/main/src/PipelineWithRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**How to use this notebook :**

This notebook intends to use a fine-tuned LLM version of LLama2 to answer questions students of UQAC may have about their school programs and classes.

- Run the load data to load Data from the github (alternatively, you can put whatever files, just create a folder called "data" in the colab session and add the files there (accepted extensions: txt, csv, doc, docx, xml, pdf, epub, hwp, ipynb, ppt))
- Run the setup which will install and import all required libraries and setup the LLM and the RAG model (loading the data files into the RAG model can take up to a couple dozens of minutes depending on the size of the data files)
- Do not run the "Test the LLM" part because it can take a few minutes and it is unnecessary (unless you want to see how it works)
- Run the Pipeline Setup
- You can then go to the Pipeline testing section and change the user_query to whatever you want and run the pipeline function

#**Load Data**

In [1]:
!mkdir data
!cd data
!git init
!git remote add origin -f https://github.com/Giraud-Pierre/DeepLearning_FineTuneLLama2Project.git
!git sparse-checkout init --cone
!git sparse-checkout set data/RAG
!git pull origin main
!cd ../
!mv data/RAG/CyclesSupérieurs.txt data/CyclesSupérieurs.txt
!mv data/RAG/PremierCycles.txt data/PremierCycles.txt
!rmdir data/RAG
!rm data/README.md
!rm README.md

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Updating origin
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 111 (delta 29), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (111/111), 7.29 MiB | 4.50 MiB/s, done.
Resolving deltas: 100% (29/29), done.
From https://github.com/Giraud-Pierre/DeepLearning_FineTuneLLama2Project
 * [new branch]      main       -> origin/main
From https://github.com

#**Setup**

##Install All the Required Packages

In [3]:
# Installs to run the LLM
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 36.6 MB/s eta 0:00:00


In [4]:
#Installs to run the RAG model
!pip install -q llama-index llama-index-embeddings-huggingface auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.6/804.6 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

##Import All the Required Libraries

In [5]:
#Import to run the LLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, PeftConfig
import torch


from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

##Import, setup and test the RAG model

In [6]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
# RAG configuration
Settings.llm = None
Settings.chunk_size = 1024 # fragment size (in chars) for the RAG
Settings.chunk_overlap = 256 # overlap between 2 adjacent fragment 4
                            # (so that no context is missing or not
                            # comprehensible because of the fragmentation)

LLM is explicitly disabled. Using MockLLM.


In [24]:
# import document in "data" directory
documents = SimpleDirectoryReader("data").load_data()

print(len(documents)) #number of documents

2


In [30]:
# Vector storing the documents (can take a few minutes depending
# on the size of the documents)
index = VectorStoreIndex.from_documents(documents)



In [33]:
# retriever configuration
top_k = 3 # number of fragments to get at each query
          # recommended: 2 to 4

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

# RAG query_engine configuration
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [34]:
# RAG testing
query = "Quelles sont les cours disponibles pour la maîtrise en informatique ?"
response = query_engine.query(query)

# Creating a context-like string that will receive the information from the RAG
# about the query
context = "Context:\n"
for item in response.source_nodes:
    # Show all the fragments selected by the RAG for being most likely
    # connected to the query
    context += item.text + "\n\n"

print(context)

Context:
La note obtenue au cours antrieur et la date d'obtention du cours dterminent la poursuite de la demande :
Aucune reconnaissance des acquis ne sera accorde pour :
les cours complts depuis 5  10 ans, avec une note infrieure  C.
les cours complts depuis plus de 10 ans.
Exprience professionnelle ou personnelle
Les personnes avec une exprience professionnelle pertinente ou avec une formation autodidacte pourront soumettre une demande de reconnaissance des acquis. Pour ceci, les professionnels devront fournir une lettre de prsentation, un CV et des lettres d'employeurs. Les personnes avec une formation autodidacte devront fournir un document expliquant leur dmarche de formation et un CV dtaill.
STRUCTURE DU PROGRAMME
Le programme comprend trente (30) crdits rpartis comme sui:
8 cours obligatoires (24 crdits)
2 cours optionnels (6 crdits)
PLAN DE FORMATION
*Les pralables sont indiqus entre parenthses  la fin du titre du cours s'il y a lieu.
COURS OBLIGATOIRES
Les huits cours suivants

##Import and setup the LLM

In [40]:
# bitsandbytes parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [41]:
# load fine-tuned model from huggingFaces
model_name = "pirroflamme/Llama2_Finetuned_DeepLearning"
model = AutoModelForCausalLM.from_pretrained(model_name,
                  quantization_config=bnb_config,
                  device_map="auto"
)

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

#**Pipeline setup**

##Prompt template

In [133]:
# prompt template (based on user query)
prompt_template_w_context = lambda context, prompt_engineering, user_query: f"""
[INST]
{context}

{prompt_engineering}

{user_query}
[/INST]
"""

##Context from RAG

In [134]:
def get_context(user_query):
  response = query_engine.query(query)
  context = "Context: \n "
  for item in response.source_nodes:
    # Show all the fragments selected by the RAG for being most likely
    # connected to the query
    context += item.text + "\n\n"

  return context

##Prompt_engineering

In [135]:
prompt_engineering =""

In [136]:
# Chain of thoughts (examples of correct question / answer)

#prompt_engineering += """ \n \n Exemples:
#Quels sont les cours disponible pour la Maîtrise en informatique ?
#answer: Les cours disponibles pour la maîtrise en informatique sont: Réseaux de neurones, Bases de données réparties, Vision artificielle et traitement des images, Informatique appliquée et optimisation, Structures de données avancées et leurs algorithmes, Interaction 3D et réalité virtuelle, Génie logiciel, Métaheuristiques en optimisation,  Architecture des applications d'entreprise, Intelligence d'affaires: principes et méthodes, Programmation sur architectures parallèles, Sécurité informatique, Systèmes intégrés de gestion d'entreprise, Gestion de projets informatiques, Programmation de plateformes mobiles, Fondamentaux de l'apprentissage automatique, Principes des moteurs de jeux, Cryptographie, Conception et architecture des systèmes d'infonuagique, Intelligence Artificielle, Gestion des incidents de sécurité, Sécurité des applications, Gestion de la cybersécurité et des données personnelles, Séminaire thématique en cybersécurité, Apprentissage profond, Séminaire thématique en Intelligence artificielle, Jeux sérieux, Séminaire thématique en jeu vidéo, Programmation réseau dans les jeux vidéo, Sécurité informatique pour l'Internet des Objets, Analyse forensique, Apprentissage automatique pour les données massives, Internet des objets, Atelier en optimisation avancée, Sujets spéciaux, Programmation objet avancée, Spécification, test et vérification, Principes de conception et de développement de jeux vidéo
#Quels programmes proposent le cours 8INF874 cryptographie ?
#answer: Les programmes proposant le cours cryptographie sont : le Programme court de deuxième cycle en informatique pour étudiants en séjour d'études, la Maîtrise en informatique (jeux vidéo), la Maîtrise en informatique (cybersécurité), la Maîtrise en informatique, la Maîtrise en informatique (professionnel), le Diplôme d'études supérieures spécialisées en informatique appliquée
#en quoi consiste le cours 8INF896 Séminaire thématique en intelligence artificielle ?
#answer : Le séminaire théùatique en intelligence artificielle permet de bénéficier d'une formation adaptée dans le domaine de l'intelligence artificielle. Le contenu est variable selon les besoins des étudiants et l'expertise professorale disponible.
#"""

# Does not work, completely breaks the model for some reason ...

In [137]:
# Do not hallucinate
prompt_engineering += """\n Reste concis et factuel. Répond "je ne suis pas sûr de la réponse" si tu n'es pas sûr
"""

In [138]:
prompt_engineering += "\n Répond à la question suivante en utilisant les informations du contexte si celui-ci est utile mais ne mentionne pas le contexte:"

In [139]:
prompt_engineering += "\n \n \n Fin du contexte \n"

## Pipeline function

In [140]:
def pipeline(user_query, prompt_engineering):
  prompt = prompt_template_w_context(get_context(user_query),prompt_engineering, user_query)
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

  response = tokenizer.batch_decode(outputs)[0]

  return response.split("Fin du contexte")[1].replace("[/INST]", "\n Réponse: ")

#**Test the LLM without RAG**

In [141]:
# Test the LLM without RAG

prompt = """[INST] Quels sont les cours disponible pour la Maîtrise en informatique ? [/INST]""" # = User query

model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s> [INST] Quels sont les cours disponible pour la Maîtrise en informatique ? [/INST] Les cours disponibles pour le programme 'Maîtrise en informatique' incluent: (3.0 cr.).</s>

* 1INFOR601 Advanced Topics in Information Science and Technology (3.0 cr.)
* 1INFOR602 Research Methods in Information Science and Technology (3.0 cr.)
* 1INFOR603 Information Science and Technology Seminar (3.0 cr.)
* 1INFOR604 Information Science and Technology Workshop (3.0 cr.)
* 1INFOR605 Information Science and Technology Project (3.0 cr.)
* 1INFOR606 Information Science and Technology Thesis (6.0 cr.)
* 1INFOR607 Information Science and Technology Internship (3.0 cr.)
* 1INFOR608 Information Science and Technology Practicum (3.0 cr.)
* 1INFOR609 Information Science and Technology Study Group (3.0 cr.)
* 1INFOR610 Information Science and Technology Independent Study (3.0 cr.)
* 1INFOR611 Information Science and Technology Topics in (3.0 cr.)
* 1INFOR612 Information Science and Technology Advanced Topics

#**Pipeline Testing (LLM with RAG)**

In [143]:
 # Change the user_query to your question
user_query = "Dans quels programmes se trouve le cours d'apprentissage profond ? "
response= pipeline(user_query,prompt_engineering)

print(response)

 


Dans quels programmes se trouve le cours d'apprentissage profond ? 

 Réponse: 
Le cours d'apprentissage profond se trouve dans le programme 7LNG238.</s>


In [ ]:
 # Check what the context is to see what is happening
get_context(user_query)

'Context: \n La note obtenue au cours antrieur et la date d\'obtention du cours dterminent la poursuite de la demande :\r\nAucune reconnaissance des acquis ne sera accorde pour :\r\nles cours complts depuis 5  10 ans, avec une note infrieure  C.\r\nles cours complts depuis plus de 10 ans.\r\nExprience professionnelle ou personnelle\r\nLes personnes avec une exprience professionnelle pertinente ou avec une formation autodidacte pourront soumettre une demande de reconnaissance des acquis. Pour ceci, les professionnels devront fournir une lettre de prsentation, un CV et des lettres d\'employeurs. Les personnes avec une formation autodidacte devront fournir un document expliquant leur dmarche de formation et un CV dtaill.\r\nSTRUCTURE DU PROGRAMME\r\nLe programme comprend trente (30) crdits rpartis comme sui:\r\n8 cours obligatoires (24 crdits)\r\n2 cours optionnels (6 crdits)\r\nPLAN DE FORMATION\r\n*Les pralables sont indiqus entre parenthses  la fin du titre du cours s\'il y a lieu.\r\n